In [ ]:
import numpy as np
import torch
from isaacgym import gymapi, gymtorch, torch_utils

In [ ]:
from env import FrankaPandaEnv

In [ ]:
env = FrankaPandaEnv(use_gpu=True, use_gpu_pipeline=False)

In [ ]:
env.reset_pose("default")
env._refresh()
for _ in range(10000):
    env._refresh()
    dpose = torch.zeros((env.num_envs, 7))
    dpose[:, :3] = torch.Tensor([0.0, 0.2, 0.05]) - env.states["eef_pos"]

    target_quat = torch.Tensor([1, 0, 0, 0])
    current_quat = env.states["eef_quat"]
    diff_quat = torch_utils.quat_mul(
        target_quat.unsqueeze(0).expand_as(current_quat),
        torch_utils.quat_conjugate(current_quat),
    )
    dpose[:, 3:7] = diff_quat

    dpose[:, -1] = -1
    env._refresh()
    env.pre_physics_step(dpose)
    env.step_physics()
    env.step_rendering()